This example notebook covers ways to generate synthetic data using `numerblox` components. Synthetic data can be a great way to improve performance simply by having more data to train. We will both cover ways to generate synthetic target variables and features.

## 0. Download and load

In [1]:
import pandas as pd
from uuid import uuid4

from numerblox.download import NumeraiClassicDownloader

In [2]:
unique_id = uuid4()

dl = NumeraiClassicDownloader(directory_path=f"synth_test_{unique_id}")
dl.download_training_data(version="4.2", int8=True)

No existing directory found at 'synth_test_ecc9c7f1-e9e8-475e-b8e4-2f15ad7cd41f'. Creating directory...


2023-09-25 14:35:28,180 INFO numerapi.utils: starting download
synth_test_ecc9c7f1-e9e8-475e-b8e4-2f15ad7cd41f/train_int8.parquet: 1.88GB [02:20, 13.4MB/s]                            


2023-09-25 14:37:49,315 INFO numerapi.utils: starting download
synth_test_ecc9c7f1-e9e8-475e-b8e4-2f15ad7cd41f/validation_int8.parquet: 2.17GB [02:22, 15.2MB/s]                            


In [3]:
dataf = pd.read_parquet(f"synth_test_{unique_id}/train_int8.parquet")

In [4]:
dataf.head(2)

,era,data_type,feature_honoured_observational_balaamite,feature_polaroid_vadose_quinze,feature_untidy_withdrawn_bargeman,feature_genuine_kyphotic_trehala,feature_unenthralled_sportful_schoolhouse,feature_divulsive_explanatory_ideologue,feature_ichthyotic_roofed_yeshiva,feature_waggly_outlandish_carbonisation,...,target_bravo_v4_20,target_bravo_v4_60,target_charlie_v4_20,target_charlie_v4_60,target_delta_v4_20,target_delta_v4_60,target_echo_v4_20,target_echo_v4_60,target_jeremy_v4_20,target_jeremy_v4_60
id,,,,,,,,,,,,,,,,,,,,,
n003bba8a98662e4,0001,train,4,2,4,4,0,0,4,4,...,0.25,0.0,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.25
n003bee128c2fcfc,0001,train,2,4,1,3,0,3,2,3,...,0.75,1.0,0.75,0.75,0.75,0.75,0.75,0.75,0.75,1.00


## 1. Synthetic target (Bayesian GMM)

First we will tackle the problem of creating a synthetic target column to improve model performance. `BayesianGMMTargetProcessor` allows you to generate a new target variable based on a given target. The preprocessor sample the target from a [Bayesian Gaussian Mixture model](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html) which is fitted on coefficients from a [regularized linear model (Ridge regression)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html).

This implementation is based on a [Github Gist by Michael Oliver (mdo)](https://gist.github.com/the-moliver/dcdd2862dc2c78dda600f1b449071c93).

In [5]:
from numerblox.targets import BayesianGMMTargetProcessor

In [6]:
dataf.head()

,era,data_type,feature_honoured_observational_balaamite,feature_polaroid_vadose_quinze,feature_untidy_withdrawn_bargeman,feature_genuine_kyphotic_trehala,feature_unenthralled_sportful_schoolhouse,feature_divulsive_explanatory_ideologue,feature_ichthyotic_roofed_yeshiva,feature_waggly_outlandish_carbonisation,...,target_bravo_v4_20,target_bravo_v4_60,target_charlie_v4_20,target_charlie_v4_60,target_delta_v4_20,target_delta_v4_60,target_echo_v4_20,target_echo_v4_60,target_jeremy_v4_20,target_jeremy_v4_60
id,,,,,,,,,,,,,,,,,,,,,
n003bba8a98662e4,0001,train,4,2,4,4,0,0,4,4,...,0.25,0.00,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.25
n003bee128c2fcfc,0001,train,2,4,1,3,0,3,2,3,...,0.75,1.00,0.75,0.75,0.75,0.75,0.75,0.75,0.75,1.00
n0048ac83aff7194,0001,train,2,1,3,0,3,0,3,3,...,0.50,0.25,0.50,0.25,0.50,0.25,0.50,0.25,0.50,0.25
n00691bec80d3e02,0001,train,4,2,2,3,0,4,1,4,...,0.75,0.50,0.75,0.75,0.50,0.50,0.75,0.50,0.50,0.50
n00b8720a2fdc4f2,0001,train,4,3,4,4,0,0,4,2,...,0.75,0.50,0.75,0.50,0.75,0.50,0.75,0.50,0.50,0.50


In [7]:
bgmm = BayesianGMMTargetProcessor()
bgmm.set_output(transform="pandas")
sample = dataf.sample(1000)
X = sample[["feature_polaroid_vadose_quinze", "feature_genuine_kyphotic_trehala"]].fillna(0.5)
y = sample["target"]
eras = sample['era']
bgmm.fit(X, y, eras=eras)
fake_target = bgmm.transform(X, eras=eras)

Generating fake target: 100%|██████████| 479/479 [00:00<00:00, 683.11it/s]


In [8]:
fake_target.head(10)

,fake_target
id,
n7630c37f7eac63e,0.50
n2822c11fdda3a05,0.50
n4bbcc611330e8c9,0.50
n7cb19c5d5278030,0.50
n183d83b326a9661,0.25
nd00493791012fd3,0.50
na9415225ee2acad,0.75
n8570f6cb08a8ae3,0.50
n8a1beeaceea745b,0.75


In [9]:
# Clean up environment
dl.remove_base_directory()

Path: '/home/clepelaars/numerblox/examples/synth_test_ecc9c7f1-e9e8-475e-b8e4-2f15ad7cd41f'
